In [1]:
import os, random
import numpy as np
import pandas as pd
from ase.io import read
from collections import Counter

In [2]:
train = read('./data/train.xyz', format='extxyz', index=':') # 전체 데이터 불러오기
test = read('./data/test.xyz', format='extxyz', index=':')
sample = pd.read_csv('./data/sample_submission.csv')

# 1. train data

## 1.1 데이터의 길이

In [24]:
print("length of train: ",len(train))
print("information of train[0]: ",train[0])
print("length of train[0]: ",len(train[0]))
print("information of train[0][0]: ",train[0][0])
print("information of train[0][1]: ",train[0][47])

length of train:  22510
information of train[0]:  Atoms(symbols='N24Si24', pbc=True, cell=[8.52238831, 8.52238831, 8.52238831], forces=..., calculator=SinglePointCalculator(...))
length of train[0]:  48
information of train[0][0]:  Atom('N', [1.59173729, 4.20048347, 7.83224513], index=0)
information of train[0][1]:  Atom('Si', [2.55477637, 4.14399683, 0.91811246], index=47)


- train에는 22510개의 Atoms가 존재한다. 이 Atoms는 여러 정보를 가지고 있다.
- 0번 Atoms는 48개의 atom을 가지고 있다. 이는 symbols = N24Si24 즉, N 24개 Si 24개로 이루어져 있는 것을 알 수 있다. 
- 이 Atom들은 index 0부터 47번까지 존재한다.

## 1.2 Atoms로부터 얻을 수 있는 정보

In [26]:
atoms = train[0]

In [126]:
print("the number of positions",type(atoms.get_positions())) # baseline에서 사용. inputs
print("the number of forces",type(atoms.get_forces())) # baseline에서 사용. label 1
print("total energy: ", atoms.get_total_energy()) # baseline에서 사용. label 2
print("potential energy: ", atoms.get_potential_energy()) # total energy와 같음
print("\n원자의 속성")
print("number of atoms: ", atoms.get_global_number_of_atoms()) # 원자의 개수
print("chemical formula:", atoms.get_chemical_formula())
print("chemical symbols:", atoms.get_chemical_symbols())
print("atomic numbers: ", atoms.get_atomic_numbers())
print("atomic masses: ", atoms.get_masses()) # 원자의 종류에 따라 다름
print("\n도움 될 수도 있는 것")
print("pbc:",atoms.get_pbc()) # 3 방향인가보다
print("cell unit: ", atoms.get_cell()) # 셀 구조의 크기. PBC인 경우 반복되는 셀의 크기를 알려줌
print("all distance of atoms: ", atoms.get_all_distances().shape)
print("moments of inertia: ",atoms.get_moments_of_inertia()) # 3개의 값을 가지게 됨 중요할수도?
print("volume: ",atoms.get_volume())
print("\n별로 쓸모 없어보이는 것")
print("initial magnetic moments: ", atoms.get_initial_magnetic_moments()) # 0만 나오는지 확인해야겠다
print("velocity: ",type(atoms.get_velocities())) # 모두 0이 나오나 확인해야겠다.
print("user custom tags: ", atoms.get_tags()) # 사용자 정의 태그라고 한다.
print("calculator: ",atoms.get_calculator())
print("center of mass:",atoms.get_center_of_mass())
print("angular momentum: ", atoms.get_angular_momentum())
print("momenta: ",atoms.get_momenta().shape) # 각 원자의 운동량(모멘텀)
print("kinetic energy: ", atoms.get_kinetic_energy())
print("temperature: ", atoms.get_temperature())

# print("stress: ", atoms.get_stress()) # 계산기에 따라서 작동하지 않을 수도 있다고 한다.
# print("magnetic moments: ", atoms.get_magnetic_moments()) # 계산기에 따라서 작동하지 않을 수도 있다고 한다.
# print("surface area:",atoms.get_surface_area()) # 안 나온다.
# print("constraint: ",atoms.get_constraint()) # 안 나온다.
# print("charges: ",atoms.get_charges()) # 안 나온다.
# print("dipole: ",atoms.get_dipole()) # 안 나온다.
# print("linear momentum: ", atoms.get_linear_momentum()) # 안 나온다.

the number of positions <class 'numpy.ndarray'>
the number of forces <class 'numpy.ndarray'>
total energy:  -320.35840439
potential energy:  -320.35840439

원자의 속성
number of atoms:  48
chemical formula: N24Si24
chemical symbols: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si', 'Si']
atomic numbers:  [ 7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14]
atomic masses:  [14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007
 14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007 14.007
 14.007 14.007 14.007 14.007 28.085 28.085 28.085 28.085 28.085 28.085
 28.085 28.085 28.085 28.085 28.085 28.085 28.085 28.085 28.085 28.085
 28.085 28.085 28.085 28.085 28.085 28.085 

In [5]:
sequence_train, symbols, positions_x, positions_y, positions_z, forces, energies = [], [], [], [], [], [], []

for i in range(len(train)):
    mole = train[i] # 각 분자 - 22510개

    atoms = len(mole) # 원자 개수 - 48개
    sequence_train.append(atoms) # 22510개의 mole에 들어있는 atoms의 개수를 저장

    position = mole.get_positions() # 원자 위치 정보 -> (48, 3)
    force = mole.get_forces() # label 1 -> (48, 3)

    energy = mole.get_total_energy() # label 2 -> float 값 하나
    energies.append(energy)

    for j in range(len(mole)): # 각 원자에 대해 반복 -> 48회
        atom = mole[j] # i번 분자의 j번째 원자

        positions_x.append(position[j][0])
        positions_y.append(position[j][1])
        positions_z.append(position[j][2])
        forces.append(force[j])

train_df = pd.DataFrame({'position_x': positions_x, 'position_y':positions_y, 'position_z':positions_z, 'force':forces}) # sequence_train, symbols, energies 아직 사용 안 함
train_df.head()

,position_x,position_y,position_z,force
0,1.591737,4.200483,7.832245,"[-1.9364797, -2.75540073, 0.90898967]"
1,5.640802,2.305094,4.606757,"[1.77046974, -0.17350153, -1.99398617]"
2,6.672786,8.483263,2.981881,"[-2.05488716, -0.29381591, -0.89173793]"
3,1.908548,0.147931,1.741693,"[-0.89207197, -0.8143158, -1.36426899]"
4,4.375650,6.837884,1.948188,"[-4.65938123, -0.77685475, -3.07403915]"


In [10]:
Counter(sequence_train)

Counter({48: 11298,
         384: 147,
         105: 2081,
         28: 4946,
         222: 27,
         278: 27,
         376: 27,
         280: 54,
         140: 27,
         168: 54,
         96: 1280,
         101: 27,
         102: 160,
         149: 160,
         64: 931,
         32: 399,
         16: 399,
         27: 399,
         214: 40,
         510: 27})